In [ ]:
%pylab inline
%config InlineBackend.figure_format = 'svg'

import pandas as pd
import seaborn as sns
import numbers
sns.set()

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV
from sklearn.metrics import r2_score

In [ ]:
!ls data/

In [ ]:
b2pagination = 'data/2-bed-manhattan-all.json'
b2page = 'data/2-bed-manhattan-pages.json'

b1pagination = 'data/1-bed-manhattan-all.json'
b1page = 'data/1-bed-manhattan-pages.json'

In [ ]:
b2p_df = pd.read_json(b2pagination)
b2u_df = pd.read_json(b2page)

b1p_df = pd.read_json(b1pagination)
b1u_df = pd.read_json(b1page)

In [ ]:
print('2o', b2p_df.shape)
print('2u', b2u_df.shape)

print('2o', b1p_df.shape)
print('2u', b1u_df.shape)

In [ ]:
b2p_df_no_dup = b2p_df.drop_duplicates(['url'], keep='first')
b2u_df_no_dup = b2u_df.drop_duplicates(['url'], keep='first')

b1p_df_no_dup = b1p_df.drop_duplicates(['url'], keep='first')
b1u_df_no_dup = b1u_df.drop_duplicates(['url'], keep='first')

In [ ]:
print('2o', b2p_df_no_dup.shape)
print('2u', b2u_df_no_dup.shape)

print('1o', b1p_df_no_dup.shape)
print('1u', b1u_df_no_dup.shape)

In [ ]:
b2_merge = pd.merge(b2p_df_no_dup, b2u_df_no_dup, on=['url'])
b1_merge = pd.merge(b1p_df_no_dup, b1u_df_no_dup, on=['url'])

In [ ]:
b2_merge.head(1)

In [ ]:
b1_merge.head(1)

In [ ]:
b2_dropped = b2_merge.drop(['furnished', 'has_broker_fee', 'sqft'], axis=1)
b1_dropped = b1_merge.drop(['furnished', 'has_broker_fee', 'sqft'], axis=1)


In [ ]:
b2_dropped.columns

In [ ]:
y_coln = 'price'

In [ ]:
x_continuous_coln = [
    'baths',
    'building_units',
    'station_nearest_distance',
    'station_total'
]

In [ ]:
x_dummies_coln = [
    'building_type',
    
    
       'amen_outdoor_space', 'amen_pets_allowed', 'amen_dishwasher',
       'amen_doorman', 'amen_elevator', 'amen_gym', 'amen_laundry_in_building',
       'amen_live_in_super', 'amen_parking', 'amen_laundry_in_unit' 
]

In [ ]:
x_dummies_coln_neighbors = [
    'building_type',
    'neighborhood',
    
       'amen_outdoor_space', 'amen_pets_allowed', 'amen_dishwasher',
       'amen_doorman', 'amen_elevator', 'amen_gym', 'amen_laundry_in_building',
       'amen_live_in_super', 'amen_parking', 'amen_laundry_in_unit' 
]

In [ ]:
x_keep_coln = [
    'beds',
    'baths',
    
    'neighborhood',
    'building_units',
    'building_type',
    
       'amen_outdoor_space', 'amen_pets_allowed', 'amen_dishwasher',
       'amen_doorman', 'amen_elevator', 'amen_gym', 'amen_laundry_in_building',
       'amen_live_in_super', 'amen_parking', 'amen_laundry_in_unit'     
]

In [ ]:
def clean(df):
    print('\nstart shape', df.shape)

    # drop these columns that dn't hav emuch data
    
    # drop building types that couldn't be specified
    dropping = df['building_type'] != -1
    df = df[dropping]
    print('\n  dropping building_type -1: ', sum(~dropping))
    
    # drop rows that have negative continuous variable values (signaling an issue during scraping)
    dropping = df['station_nearest_distance'] != -1
    df = df[dropping]
    print('\n  dropping station_nearest_distance -1: ', sum(~dropping))
    
    dropping = df['station_nearest_total'] != -1
    df = df[dropping]
    print('\n  dropping station_nearest_total -1: ', sum(~dropping))  
    
    dropping = df['building_stories'] != -1
    df = df[dropping]
    print('\n  dropping building_stories -1: ', sum(~dropping))  

    dropping = df['building_units'] != -1
    df = df[dropping]
    print('\n  dropping building_units -1: ', sum(~dropping))  

    
    print('\nend shape', df.shape)
    
    return df
    
    
    

In [ ]:
b2_clean = clean(b2_dropped)
b1_clean = clean(b1_dropped)

In [ ]:
def check_neg_values(df):
    print('\n')
    for col in df:
        if isinstance(df[col].iloc[0], numbers.Real):
            summation = sum(df[col] < 0)
            print(col, summation)


In [ ]:
def check_valid_values(df):
    print('\n')
    for col in df:
        print(col)
        df[col].sort_values()

In [ ]:
def remove_outlier(df, col='price', threshold=1.5):
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    lowest_limit = df[col] >= q1 - (threshold * iqr)
    highest_limit = df[col] <= q3 + (threshold * iqr)
    return df[lowest_limit & highest_limit]    


In [ ]:
plt.boxplot(b2_clean[y_coln])
plt.savefig('boxplot-outliers.png')

In [ ]:
check_valid_values(b2_clean)


In [ ]:
check_valid_values(b1_clean)

In [ ]:
check_neg_values(b2_clean)

In [ ]:
check_neg_values(b1_clean)

In [ ]:
b2_clean

In [ ]:
df2_out = remove_outlier(b2_clean, y_coln)
df1_out = remove_outlier(b1_clean, y_coln)



In [ ]:
df2_out

In [ ]:
plt.boxplot(df2_out[y_coln])
plt.savefig('boxlplot-no-outlier.png')

In [ ]:
pickle_bed1 = 'data/clean_1-bed.pkl'
pickle_bed2 = 'data/clean_2-bed.pkl'


In [ ]:
df2_out.to_pickle(pickle_bed2)
df1_out.to_pickle(pickle_bed1)


In [ ]:
dfb1 = pd.read_pickle(pickle_bed1)
dfb2 = pd.read_pickle(pickle_bed2)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dfb2, dfb2[y_coln], test_size=0.2, random_state=42)

In [ ]:
df_X_cols = ['beds', 'baths']
dfs_X_cols = ['beds', 'baths', 'sqft'] # furnished has None values
dfb1_X_cols = ['baths']
dfb2_X_cols = ['baths']
dfb3_X_cols = ['baths']
dummies = ['neighborhood']

In [ ]:
def add_dummies(df, y_col, x_cols, dummy_cols):
            
    dummy_df = pd.get_dummies(df, columns=dummy_cols, drop_first=True)
    
    agg_dummy_col_names = []
    
    for col_name in dummy_cols:
        dummy_col_names = [col for col in dummy_df.columns if col_name + '_' in col]
        agg_dummy_col_names += dummy_col_names
    
    cols = x_cols + agg_dummy_col_names
    return (dummy_df, cols)

In [ ]:
def split_columns_val(df, y_col, x_cols, dummy_cols):
    """
    Take dataframe and add dummy variables
    
    """
    
    dummy_df, cols = add_dummies(df, y_col, x_cols, dummy_cols)
    
    return split_and_validate(dummy_df[cols], df[y_col])

In [ ]:
def split_and_validate(X, y, test='lr'):
    """
    Take dataframe with dummy variable and run regression model
    """

    # perform train/val split
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # fit linear regression to training data
    if test == 'lr':
        lr_model = LinearRegression()
        lr_model.fit(X_train, y_train)

        # score fit model on validation data
        val_score = lr_model.score(X_val, y_val)        

    # report results
    print('\nValidation R^2 score was:', val_score)
    print('Feature coefficient results: \n')
    zipped = zip(X.columns, lr_model.coef_)
    for feature, coef in zipped:
        print(feature, ':', f'{coef:.2f}')
    
    return (X_train, X_val, y_train, y_val, lr_model)

In [ ]:
X_train.info()

In [ ]:
"""
Trying including and excluding neighborhoods some amenities by changing values from x_dummies_coln.
"""

X_sample_train, X_sample_val, y_sample_train, y_sample_val, lr_model = split_columns_val(X_train, 'price', x_continuous_coln, x_dummies_coln)

In [236]:
print('training score', lr_model.score(X_sample_train, y_sample_train))
print('testing score', lr_model.score(X_sample_val, y_sample_val))


training score 0.6838318519764027
testing score 0.7177806244606443


In [ ]:
x_test_with_dummies = add_dummies(X_test)
y_hat = lr_model.predict(x_test_with_dummies)

In [ ]:
plt.scatter(y_val, y_hat)
plt.xlabel('Actual rental unit prices')
plt.ylabel('Predicted rental unit prices')
plt.title('Rental Prices for 2 bd Units')